<a href="https://colab.research.google.com/github/shevtsovasofi/machine_learning/blob/main/attempt2_english_shevtsovaipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание

Генерация текста на основе небольшого датасета

- Предварительный анализ: чистка текста
- Обучение модели. Используйте образец из туториала по RNNи
- Генерация текста. Используйте образец из туториала по RNN
- Сгенерируйте несколько текстов с помощью созданной модели

In [25]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import requests
import re

In [26]:
url = "https://www.gutenberg.org/files/11/11-0.txt"  #я взяла текст "Алисы в стране чудев" из проекта Гутенберг
data = requests.get(url).text

In [27]:
data = data.lower()
data = re.sub(r'[^a-z\s\.]', '', data)  # Оставим точки для разделения
data = re.sub(r'\s+', ' ', data)

In [28]:
sentences = data.split('.') #разделяем на предложения

In [29]:
sentences = [s.strip() for s in sentences if s.strip() != '']

for i, s in enumerate(sentences[:50]):
    print(f"{i+1}: {s}")

1: start of the project gutenberg ebook illustration alices adventures in wonderland by lewis carroll the millennium fulcrum edition
2: contents chapter i
3: down the rabbithole chapter ii
4: the pool of tears chapter iii
5: a caucusrace and a long tale chapter iv
6: the rabbit sends in a little bill chapter v
7: advice from a caterpillar chapter vi
8: pig and pepper chapter vii
9: a mad teaparty chapter viii
10: the queens croquetground chapter ix
11: the mock turtles story chapter x
12: the lobster quadrille chapter xi
13: who stole the tarts chapter xii
14: alices evidence chapter i
15: down the rabbithole alice was beginning to get very tired of sitting by her sister on the bank and of having nothing to do once or twice she had peeped into the book her sister was reading but it had no pictures or conversations in it and what is the use of a book thought alice without pictures or conversations so she was considering in her own mind as well as she could for the hot day made her feel 

In [31]:
data= sentences[15:8000]
'''
сначала я обучала на начале датасета с самой первой строчке, и он замусорился названиями глав и римскими цифрами,
которые сстабильно всплывали в генерируемых предложениях. поэтому я убрала первые 15 строк
'''

In [32]:
# Инициализируем токенизатор
tokenizer = Tokenizer()

# Обучаем токенизатор на заголовках
tokenizer.fit_on_texts(data)

# Преобразуем заголовки в последовательности чисел
sequences = tokenizer.texts_to_sequences(data)

# Создаем входные и выходные данные
X = []
y = []
for seq in sequences:
    for i in range(1, len(seq)):
        X.append(seq[:i])
        y.append(seq[i])


In [33]:
X[:10], y[:10]

([[56],
  [56, 13],
  [56, 13, 130],
  [56, 13, 130, 24],
  [56, 13, 130, 24, 27],
  [56, 13, 130, 24, 27, 1051],
  [56, 13, 130, 24, 27, 1051, 11],
  [56, 13, 130, 24, 27, 1051, 11, 14],
  [56, 13, 130, 24, 27, 1051, 11, 14, 825],
  [56, 13, 130, 24, 27, 1051, 11, 14, 825, 66]],
 [13, 130, 24, 27, 1051, 11, 14, 825, 66, 10])

In [34]:
# Преобразуем списки в массивы numpy
X = np.asarray(X, dtype="object")
y = np.array(y)

# Дополняем последовательности до одинаковой длины
X = pad_sequences(X)

# Преобразуем y в one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=len(tokenizer.word_index) + 1)

In [35]:
# Создаем модель
model = Sequential()

# Добавляем слой Embedding
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=X.shape[1]))

# Добавляем слой LSTM
model.add(LSTM(150, return_sequences=False))

# Добавляем полносвязный слой
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))

# Компилируем модель
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Выводим информацию о модели
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [36]:
history = model.fit(X, y, epochs=50, batch_size=64, validation_split=0.2) #это самые хорошие результаты, которых удалось достичь...

Epoch 1/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 9s 25ms/step - accuracy: 0.0473 - loss: 6.5940 - val_accuracy: 0.0852 - val_loss: 6.1841
Epoch 2/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - accuracy: 0.0560 - loss: 5.9215 - val_accuracy: 0.0994 - val_loss: 6.1579
Epoch 3/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - accuracy: 0.0724 - loss: 5.7071 - val_accuracy: 0.0979 - val_loss: 6.0203
Epoch 4/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - accuracy: 0.0979 - loss: 5.4128 - val_accuracy: 0.1163 - val_loss: 5.9538
Epoch 5/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.1183 - loss: 5.1794 - val_accuracy: 0.1232 - val_loss: 5.9178
Epoch 6/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.1319 - loss: 4.9576 - val_accuracy: 0.1289 - val_loss: 5.8958
Epoch 7/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 0.1499 - loss: 4.7445 - val_accuracy: 0.1289 - val_loss: 5.8883
Epoch 8/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 0.1675 - loss: 4.5771 - val_a

In [37]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 298, 100)            │         274,700 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 150)                 │         150,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2747)                │         414,797 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,520,293 (9.61 MB)

 Trainable params: 840,097 (3.20 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,680,196 (6.41 MB)

In [38]:
# Функция для генерации текста
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [44]:
# Генерируем новый текст
generated_text = generate_text("cat said", 20, X.shape[1])
print(generated_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
cat said the cat and vanished again in a low voice to the company generally you are all pardoned such a tiny


# Вывод:  loss большой, но на примерах сгенерированного текста видно, на чем модель обучалась, также соблюдается какая-то грамматическая структура.